In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# ensemble 위한 분류학습기 추가하기

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler , Binarizer 
from sklearn.model_selection import train_test_split , GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_curve , roc_curve

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as ms
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer

In [7]:
cancer=load_breast_cancer()
cancer_df=pd.DataFrame(cancer.data,columns=cancer.feature_names)
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from IPython.display import Image
Image('./images/ensemble-01.png')

In [ ]:
from IPython.display import Image
Image('./images/ensemble-01.png')

In [14]:
# 소프트 보팅
# 앙상블을 구현하는 보팅 분류기 -VotingClassifier

#학습기생성
dtc=DecisionTreeClassifier()
lr=LogisticRegression()
knn=KNeighborsClassifier()

# 앙상블 모델을 구현하기위해서 구현을 위한 분류기(voting)
voting_model=VotingClassifier([('dtc',dtc),('lr',lr),('knn',knn)],voting='soft')
# 데이터 세트를 분리
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    test_size=.2,
                                                    random_state=156)

# 하드보팅 다수결  !!!!!!!!! 소프트보팅 평균 !!!!!!!!!!!!!!!
# 보팅 분류기를 이용해서 학습/예측/평가
# 학습을 시킨다는 voting모델을 이용하는 것
voting_model.fit(X_train,y_train)
y_pred=voting_model.predict(X_test)

print('Voting accuracy :',accuracy_score(y_test,y_pred))
print()
# 개별 학습기에 대한 학습/예측/평가
classifier=[dtc,lr,knn]
for c in classifier:
    c.fit(X_train,y_train)
    pred=c.predict(X_test)
    print(c.__class__.__name__,accuracy_score(y_test,pred))

Voting accuracy : 0.9385964912280702

DecisionTreeClassifier 0.9473684210526315
LogisticRegression 0.9385964912280702
KNeighborsClassifier 0.9035087719298246


In [19]:
# RandomForestClassifier
rfc=RandomForestClassifier()
grid_params={
    'max_depth':[2,4,6],
    'min_samples_split':[3,6,9],
    'min_samples_leaf':[2,4,6]
}
gsRFC=GridSearchCV(rfc,grid_params,cv=5 ,scoring='accuracy')
gsRFC.fit(X_train,y_train)
RFC_best=gsRFC.best_estimator_
print(RFC_best)
gsRFC.best_score_

RandomForestClassifier(max_depth=4, min_samples_leaf=2, min_samples_split=3)


0.9582417582417582